# Get $\omega_0$ and added mass from "B" models

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

In [ ]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 15, 5

import rolldecay.database as database
from mdldb.tables import Run
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.direct_estimator_cubic import EstimatorCubic, EstimatorQuadratic, EstimatorLinear

from rolldecayestimators import equations
from rolldecayestimators import symbols
from rolldecayestimators.substitute_dynamic_symbols import lambdify,run
import sympy as sp


import data
import copy
from sklearn.pipeline import Pipeline
import seaborn as sns

from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex

## Quadratic model
Has quadratic damping and stiffness:

In [ ]:
Math(vlatex(EstimatorQuadratic.roll_decay_equation))

In [ ]:
df_quadratic = database.load(rolldecay_table_name='rolldecay_quadratic_b', limit_score=0.95)

mask = ((df_quadratic['gm'] > 0) & (df_quadratic['Volume'] > 0))
df_quadratic=df_quadratic.loc[mask].copy()

rho=1000
g=9.81
scale_factor=df_quadratic['scale_factor']

df_quadratic.rename(columns = {'KXX':'kxx', 'gm':'GM'}, inplace=True)

df_quadratic['kxx']/=scale_factor
df_quadratic['beam']/=scale_factor
df_quadratic['GM']/=scale_factor
df_quadratic['g'] = g
df_quadratic['rho'] = rho
df_quadratic['Volume']/=scale_factor**3
df_quadratic['m'] = df_quadratic['Volume']*rho
df_quadratic['Ixx'] = df_quadratic['kxx']**2*df_quadratic['m']

In [ ]:
meta_data = df_quadratic.iloc[0]
run_id = int(meta_data['id'])
db = database.get_db()
db_run = db.session.query(Run).get(run_id)
df = database.load_run(db_run, save_as_example=True)

In [ ]:
df.plot(y='phi')

In [ ]:
lowpass_filter = LowpassFilterDerivatorTransformer(cutoff=2, minimum_score=0.99)
#scaler = ScaleFactorTransformer(scale_factor=db_run.model.scale_factor)  # dummy value None for now
cutter = CutTransformer(phi_max=np.deg2rad(9), phi_min=np.deg2rad(0.25))
offset_transformer = OffsetTransformer()

steps = [
    ('filter',lowpass_filter),
#    ('scaler',scaler),  # Is froude scaling a good idea??
    ('cutter', cutter), 
    ('offset_transformer',offset_transformer)
]
        
preprocessor = Pipeline(steps) # define the pipeline object.
preprocessor.fit(df)
X = preprocessor.transform(df)

In [ ]:
X.plot(y='phi')

In [ ]:
meta_data = df_quadratic.loc[run_id]
quadratic_estimator = EstimatorQuadratic.load(data=meta_data, X=X)
quadratic_estimator.plot_fit()

In [ ]:
equations.C_equation_quadratic

In [ ]:
stiffness_lambda = lambdify(-sp.solve(equations.C_equation_quadratic,symbols.C)[0]*symbols.phi)

In [ ]:
X['restoring moment'] = stiffness_lambda(C_1=quadratic_estimator.parameters['C_1A'], 
                 C_3=quadratic_estimator.parameters['C_3A'], phi=X['phi'])

X['restoring moment linear'] = stiffness_lambda(C_1=quadratic_estimator.parameters['C_1A'], 
                 C_3=0, phi=X['phi'])

In [ ]:
X.plot(y=['restoring moment','restoring moment linear'])

In [ ]:
X.plot(x='phi',y=['restoring moment','restoring moment linear'])

In [ ]:
Math(vlatex(equations.restoring_equation_linear))

In [ ]:
equations.normalize_equations[symbols.C_1]

In [ ]:
equations.C_equation_linear

In [ ]:
C_1_equation = equations.C_equation_linear.subs(symbols.C,symbols.C_1)
C_1_equation

In [ ]:
eqs = [
    C_1_equation,
    equations.normalize_equations[symbols.C_1]
]

C_1A = equations.new_symbols[symbols.C_1]

A44_equation = sp.Eq(symbols.A_44,sp.solve(eqs,symbols.C_1,symbols.A_44)[symbols.A_44])
A44_equation

In [ ]:
A44_lambda = lambdify(sp.solve(A44_equation,symbols.A_44)[0])

In [ ]:
def calculate_A44(meta_data):
    return run(function = A44_lambda, inputs=meta_data)
    return added_mass

def calculate_kxx_ratio(meta_data):
    A44 = calculate_A44(meta_data)
    kxx_tot = np.sqrt(A44/meta_data['m'])
    return (kxx_tot-meta_data['kxx'])/meta_data['kxx']
    

In [ ]:
meta_data = meta_data.copy()
calculate_kxx_ratio(meta_data)

In [ ]:
df_quadratic['kxx_ratio'] = df_quadratic.apply(func=calculate_kxx_ratio, axis=1)
mask = pd.notnull(df_quadratic['kxx_ratio'])
results = df_quadratic.loc[mask].copy()

In [ ]:
fig,ax=plt.subplots()
results['kxx_ratio'].hist(bins=60, ax=ax)
ax.set_xlabel(r'$\frac{k_{xxtot}-k_{xx}}{k_{xx}}$')
ax.set_title('Histogram of added roll inertia');

I have looked at the meta data from the reports for the outliers to the right and cannot find anything wrong. Strange...

In [ ]:
eqs = [equations.C_equation_linear,
       equations.omega0_equation,
      ]
omgea0_equation = sp.Eq(symbols.omega0,sp.solve(eqs,symbols.C,symbols.omega0)[0][1])
omgea0_equation

In [ ]:
omega0_lambda = lambdify(sp.solve(omgea0_equation,symbols.omega0)[0])

In [ ]:
def calculate_omega0(meta_data):
    A44 = calculate_A44(meta_data)
    return omega0_lambda(A_44=A44, GM=meta_data['GM'], g=g, m=meta_data['m'])

In [ ]:
meta_data['omega0'] = calculate_omega0(meta_data)
meta_data['omega0']

In [ ]:
quadratic_estimator.omega0

In [ ]:
quadratic_estimator.calculate_amplitudes_and_damping()
quadratic_estimator.X_amplitudes.plot(x='phi',y='omega0')

In [ ]:
results['omega0'] = results.apply(func=calculate_omega0, axis=1)

In [ ]:
fig,ax=plt.subplots()
results['omega0'].hist(bins=60, ax=ax)
ax.set_title(r'Histogram of $\omega_0$');
ax.set_xlabel('$\omega_0$');

In [ ]:
equations.omega_hat_equation

In [ ]:
omega_hat_equation = equations.omega_hat_equation.subs(symbols.omega,symbols.omega0)
omega_hat_equation

In [ ]:
omega_hat_lambda = lambdify(sp.solve(omega_hat_equation,symbols.omega_hat)[0])

In [ ]:
results['omega0_hat'] = run(omega_hat_lambda,results)

In [ ]:
fig,ax=plt.subplots()
results['omega0_hat'].hist(bins=60, ax=ax)
ax.set_title(r'Histogram of $\hat{\omega_0}$');
ax.set_xlabel('$\hat{\omega_0}$')

In [ ]:
mask = ((results['kxx_ratio'] < results['kxx_ratio'].quantile(0.005)) |  
 (results['kxx_ratio'] > results['kxx_ratio'].quantile(0.99)))
outliers = results.loc[mask].copy()
outliers['A44']=run(function=A44_lambda,inputs=outliers)
outliers[['kxx_ratio','GM','m','omega0','omega0_hat','kxx','Ixx','A44','score','model_number','project_number']]

In [ ]:
meta_data = outliers.iloc[1]
run_id = int(meta_data['id'])
db = database.get_db()
db_run = db.session.query(Run).get(run_id)
df = database.load_run(db_run, save_as_example=True)
X = preprocessor.transform(df)
quadratic_estimator = EstimatorQuadratic.load(data=meta_data, X=X)
quadratic_estimator.plot_fit()

In [ ]:
X['restoring moment'] = stiffness_lambda(C_1=quadratic_estimator.parameters['C_1A'], 
                 C_3=quadratic_estimator.parameters['C_3A'], phi=X['phi'])

X['restoring moment linear'] = stiffness_lambda(C_1=quadratic_estimator.parameters['C_1A'], 
                 C_3=0, phi=X['phi'])

In [ ]:
X.plot(x='phi',y=['restoring moment','restoring moment linear'])

In [ ]:
print(meta_data.project_path)

In [ ]:
meta_data.name

In [ ]:
temp = database.load(rolldecay_table_name='rolldecay_quadratic_b', limit_score=0.95)
temp.loc[meta_data.name]

## Correct wrong meta data

In [ ]:

db_run = db.session.query(Run).get(int(16501))
loading_condition = db_run.loading_condition
print(loading_condition.TA)
print(loading_condition.TF)

loading_condition.gm = 25.12-22.6
#loading_condition.lcg = 
#loading_condition.kg = 
loading_condition.KXX = 17.3
#loading_condition.KYY = 
#loading_condition.KZZ = 

db.session.merge(loading_condition)
db.session.commit()

In [ ]:

db_run = db.session.query(Run).get(int(18463))
loading_condition = db_run.loading_condition
print(loading_condition.TA)
print(loading_condition.TF)

#loading_condition.gm = 2.20
#loading_condition.lcg = 
#loading_condition.kg = 
loading_condition.KXX = 15.77
#loading_condition.KYY = 
#loading_condition.KZZ = 

db.session.merge(loading_condition)
db.session.commit()

In [ ]:
db_run = db.session.query(Run).get(int(16501))
loading_condition = db_run.loading_condition
print(loading_condition.TA)
print(loading_condition.TF)

#loading_condition.gm = 2.20
#loading_condition.lcg = 
#loading_condition.kg = 
loading_condition.KXX = 0.36*48
#loading_condition.KYY = 
#loading_condition.KZZ = 

db.session.merge(loading_condition)
db.session.commit()

In [ ]:
db_run = db.session.query(Run).get(int(5511))
loading_condition = db_run.loading_condition
print(loading_condition.TA)
print(loading_condition.TF)

loading_condition.gm = 2.20
#loading_condition.lcg = 
#loading_condition.kg = 
#loading_condition.KXX = 15.19 
#loading_condition.KYY = 
#loading_condition.KZZ = 

db.session.merge(loading_condition)
db.session.commit()

In [ ]:
db_run = db.session.query(Run).get(int(18405))
loading_condition = db_run.loading_condition
print(loading_condition.TA)
print(loading_condition.TF)

loading_condition.gm = 8.02
#loading_condition.lcg = 
#loading_condition.kg = 
loading_condition.KXX = 15.19 
#loading_condition.KYY = 
#loading_condition.KZZ = 

db.session.merge(loading_condition)
db.session.commit()

## Correct wrong meta data

In [ ]:
db_run = db.session.query(Run).get(int(16490))
loading_condition = db_run.loading_condition
print(loading_condition.TA)
print(loading_condition.TF)

loading_condition.KXX = 0.36*48
db.session.merge(loading_condition)
db.session.commit()

## Correct wrong meta data

In [ ]:
db_run = db.session.query(Run).get(int(9453))
loading_condition = db_run.loading_condition
print(loading_condition.TA)
print(loading_condition.TF)

loading_condition.gm = 7.44
loading_condition.lcg = 3.80
loading_condition.kg = 8.55
loading_condition.KXX = 11.9
loading_condition.KYY = 46.6
loading_condition.KZZ = 46.7
db.session.merge(loading_condition)
db.session.commit()
